In [4]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [36]:
house = pd.read_csv('/Users/nathalie.willems/Documents/AAAM/machine_learning/AI4Environment/solar_home_system_data_log-master/data/House_2_January_TimeSeries.csv')
house2 = pd.read_csv('/Users/nathalie.willems/Documents/AAAM/machine_learning/AI4Environment/solar_home_system_data_log-master/data/House_3_January_TimeSeries.csv')
house3 = pd.read_csv('/Users/nathalie.willems/Documents/AAAM/machine_learning/AI4Environment/solar_home_system_data_log-master/data/House_4_January_TimeSeries.csv')

house2.head(5)



,Time,#0: CH0 [Â°C],#1: CH1 [V],#2: CH2 [Digit],#3: CH3 [V],#4: CH4 [Digit],PV Current,Battery Current
0,6/23/15 12:28,30.40,18.37,32777,12.89,32779,-0.114042,-0.09240
1,6/23/15 12:29,30.42,18.95,32776,12.88,32779,-0.095035,-0.09240
2,6/23/15 12:30,30.43,18.79,32774,12.88,32777,-0.057021,-0.05544
3,6/23/15 12:31,30.43,13.30,32760,12.80,32750,0.209077,0.44352
4,6/23/15 12:32,30.42,13.23,32766,12.75,32745,0.095035,0.53592


In [42]:

columns = ['Time', 'Temperature', 'PV_V', 'Digit', 'Battery_V', 'Digit2', 'PV_Current', 'Battery_Current']
house.columns = columns
house2.columns = columns
house3.columns = columns

In [43]:
# Convert to time values 
house['Time'] = pd.to_datetime(house['Time'])
house_time = house.set_index(house['Time'])
house_time.drop('Time', inplace=True, axis=1)

house2['Time'] = pd.to_datetime(house2['Time'])
house2_time = house2.set_index(house2['Time'])
house2_time.drop('Time', inplace=True, axis=1)

house3['Time'] = pd.to_datetime(house3['Time'])
house3_time = house3.set_index(house3['Time'])
house3_time.drop('Time', inplace=True, axis=1)

In [ ]:

#Values of slope and zero are based on experimental data calibrating
#digital readings of current to current values in Amps

# calibration slope for 3 amps
slope_channel_2 = [ -0.0190226, -0.0181098, -0.018034, -0.0184049, 0.0185455]
slope_channel_4 = [ 0.017730, -0.01779, -0.01845, -0.018519, 0.018765]

zero_channel_2 = [32766, 32786, 32776, 32771, 32774]
zero_channel_4 = [32775, 32785, 32777, 32771, 32775]

house_number = 2

house['PV_Current_Adj'] = (house['PV_Current'] - zero_channel_2[house_number])*slope_channel_2[house_number]
house['Battery_Current_Adj'] = (house['Battery_Current'] - zero_channel_4[house_number])*slope_channel_4[house_number]

# Set negative PV current values to 0 

house['PV_Current_Adj'] = np.where(house['PV_Current_Adj'] < 0, 0, house['PV_Current_Adj'])

In [47]:
# Calculate load values 

house['PV_Power'] = house['PV_V'] * house['PV_Current']
house['Battery_Power'] = house['Battery_V'] * house['Battery_Current']
house['Load'] = house['PV_Power'] + house['Battery_Power']

# Set negative load values to 0 
house['Load_Corrected'] = np.where(house['Load'] < 0, 0, house['Load'])


In [66]:
# Calculate Rough Proxy for Irradiance

# Assumptions 
pv_eff = .15 # efficiency
pv_power = 70 # watts, TPS panel
pv_area = .467 # area of panel m2
cc_eff = 0.95 # charge controller efficiency

house['Solar_Irradiance'] = (house['PV_Power'] / (pv_eff * pv_area)) * (1/cc_eff)
house.head(5)

# Separate Spillage from Load 
# Only day-time appliance 
fan_spillage = 14 # %mean power consumption of fan, Watts 
sigma_fan_spillage = .1*fan_spillage # assumption on standard deviation of fan power
ci_90 = 1.645
ci_95 = 1.96 

# 90% confidence interval 
fan_90_ci_lower = fan_spillage-ci_90*sigma_fan_spillage 
fan_90_ci_upper = fan_spillage + ci_90*sigma_fan_spillage

# if solar irrdiance is above 5 W/m2 and load value is NOT in 90% CI range, classify as spillage

house['Spillage'] = np.where((house['Solar_Irradiance'] > 0.05) & (house['Load_Corrected'] < fan_90_ci_lower) | (house['Load_Corrected'] > fan_90_ci_upper), house['Load_Corrected'], 0)
house['Spillage'] = np.where(house['Load_Corrected'] < fan_90_ci_upper, house['Load_Corrected'], 0)

house['Time'] = pd.to_datetime(house['Time'])
house_time = house.set_index(house['Time'])
house_time.drop('Time', inplace=True, axis=1)


In [ ]:
plt.figure(figsize=(16,8))
# ax1 = plt.subplot(121)
# ax2 = plt.subplot(122)
house_time.loc[:,['Load_Corrected', 'Spillage']].plot(kind='line')
#house2_time.loc[:,['Battery_Current','Battery_V']].plot(ax=ax2, kind='line')
plt.show()